- https://www.youtube.com/watch?v=9HhcFiSgLok&t=12s
- https://github.com/Coding-Crashkurse/LangGraph-Tutorial/blob/main/agent_team.ipynb

# 계층형 에이전트 시스템 - 심사 진행 방식 에이전트 구축 방법

## 소개
LangGraph를 사용하여 계층형 다중 에이전트 시스템을 만드는 방법을 설명합니다.
작성된 뉴스를 평가하여 게시 여부를 판단하는 시스템입니다.
## Agent System 구성
- 여러 자율 에이전트로 구성된 시스템을 만듭니다.
- 각 에이전트의 핵심 부분은 LLM(대형 언어 모델)입니다.
- 데스크 에이전트: 기사 내용이 축구 이적 관련 뉴스 기사인지 판단합니다.
- 편집자: 기사가 번역되거나 재작성되어야 하는지 결정합니다. 기사가 적절하면 게시자로 넘깁니다.
- 번역자와 확장자는 편집자 에게만 보고하며, 게시자에게 직접 전달할 권한은 없습니다.

Team of Agents with a supervisor

LangSmith 설정

In [ ]:
# LangSmith 설정

import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__708b8970829247d1a055f33c434aad1d"
os.environ["LANGCHAIN_PROJECT"]="edu-langchain-0326"

LLM 객체 생성

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# LLM 객체 생성 함수
def Get_LLM():
    os.environ["AZURE_OPENAI_API_KEY"] = '352a6bee97b5451ab5866993a7ef4ce4'
    os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://aoai-spn-krc.openai.azure.com/'

    return AzureChatOpenAI(  
        api_version = '2024-02-01',
        azure_deployment = 'gpt-4o-kr-spn',
        temperature = 0.0
        )

### 데스크 Agent
1. 구현:
    - 축구선수 이적에 대한 기사의 관련성을 평가하는 구조화된 출력을 사용합니다.
    - AzureChatOpenAI 클래스를 사용하여 LLM에 시스템 메시지와 인간 메시지를 전달합니다.
2. 예제:
    - 메시의 팀 이적 소문을 예로 들어 평가를 수행합니다.
    - 평가 결과: "yes" (관련 있음).


구조화된 축구 이적 기사 여부 관련성 평가 결과 출력

In [ ]:
# 구조화된 LLM 결과 출력 형식 정의 클래스
class TransferNewsGrader(BaseModel):
    # 축구 이적 뉴스의 관련성 확인을 위한 이진 점수.
    """Binary score for relevance check on football transfer news."""

    # 이 기사는 축구 이적에 관한 것입니다, 'yes' 또는 'no'
    binary_score: str = Field(
        description="The article is about football transfers, 'yes' or 'no'"
    )

LLM 체인 생성
- 입력된 기사가 축구 이적 관련 뉴스 여부 심사

In [ ]:
# 축구 이적 관련 뉴스 기사 여부 평가
llm = Get_LLM()
# LLM 결과를 TransferNewsGrader 클래스 정의에 따라 평가 결과를 출력하는 기능 
structured_llm_grader = llm.with_structured_output(TransferNewsGrader)

# 시스템 프롬프트:
# 당신은 뉴스 기사가 축구 이적에 관한 것인지 평가하는 평가자입니다.
# 기사가 구단 간의 선수 이적, 잠재적인 이적 또는 확정된 이적을 명시적으로 언급하는지 확인하십시오.
# 뉴스가 축구 이적에 관한 것이라면 이진 점수 'yes'를, 아니라면 'no'를 제공하십시오.
system = """You are a grader assessing whether a news article concerns a football transfer. \n
    Check if the article explicitly mentions player transfers between clubs, potential transfers, or confirmed transfers.
    Provide a binary score 'yes' or 'no' to indicate whether the news is about a football transfer."""

# 프롬프트 생성
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "News Article:\n\n {article}")
    ]
)

# LLM 체인 생성
evaluator = grade_prompt | structured_llm_grader

# LLM 테스트
# result = evaluator.invoke({"대한축구협회(KFA)에 따르면 홍 감독은 세계 축구 흐름 파악 및 분석에 도움이 될 외국인 코치 후보군을 체크하고, 유럽에서 직접 면담을 진행할 예정이다."})
# result = evaluator.invoke({"아르헨티나 대표팀은 15일 오전 10시(한국시간) 미국 플로리다 주 마이애미의 하드 록 스타디움에서 남미축구연맹(CONMEBOL) 코파 아메리카 2024 결승전에서 콜롬비아 대표팀과 연장전까지 가는 혈투 끝에 1-0으로 승리했다."})
# result = evaluator.invoke({"영국 매체 '풋볼 365'는 AI(인공지능)가 예측한 여름 이적시장 결과를 짚으며 손흥민이 스페인 프로축구 프리메라리가 아틀레티코 마드리드로 향할 수 있다는 가능성을 조명했다."})
result = evaluator.invoke({"메시가 레알 마드리드에서 FC 바르셀로나로 이적할 것이라는 소문이 있습니다."})
result.binary_score

### 편집자
1. 책임:
    - 기사의 게시 준비 상태를 평가합니다.
    - 200단어 이상, 선정적, 한국어로 작성된 기사만 게시 가능.
    - 조건을 충족하지 않으면 확장자 또는 번역자에게 전달됩니다.
2. 구현:
    - 시스템 메시지와 구조화된 출력을 사용하여 기사 평가.

구조화된 뉴스 기사 심사 결과 출력 정의

In [ ]:
# 뉴스 기사 심사 결과 출력 정의 클래스
class ArticlePostabilityGrader(BaseModel):
    # 뉴스 기사의 게시 가능 여부, 단어 수, 선정성, 언어 확인을 위한 이진 점수.
    """Binary scores for postability check, word count, sensationalism, and language verification of a news article."""

    can_be_posted: str = Field(
        # 이 기사는 게시할 준비가 되었습니다, 'yes' 또는 'no'.
        description="The article is ready to be posted, 'yes' or 'no'"
    )
    meets_word_count: str = Field(
        # 이 기사는 최소 200단어 이상입니다, 'yes' 또는 'no'.
        description="The article has at least 200 words, 'yes' or 'no'"
    )
    is_sensationalistic: str = Field(
        # 이 기사는 선정적인 스타일로 작성되었습니다, 'yes' 또는 'no'.
        description="The article is written in a sensationalistic style, 'yes' or 'no'"
    )
    is_language_korean: str = Field(
        # 이 기사의 언어는 한국어입니다, 'yes' 또는 'no'.
        description="The language of the article is Korean, 'yes' or 'no'"
    )

LLM 체인 생성
- 기사로서 게시할 수 있는 내용인가?
- 기사가 최소 200단어를 충족하는가?
- 선정적인 내용인가?
- 한국어로 작성되었는가?

In [ ]:
# 기사 내용 심사 담당 LLM
llm_postability = Get_LLM()
# LLM 결과를 ArticlePostabilityGrader 클래스 정의에 따라 평가 결과를 출력하는 기능 
structured_llm_postability_grader = llm_postability.with_structured_output(ArticlePostabilityGrader)

# 시스템 프롬프트:
# 당신은 뉴스 기사가 게시할 준비가 되었는지, 최소 200단어를 충족하는지, 선정적인 스타일로 작성되었는지, 
# 그리고 한국어로 작성되었는지를 평가하는 평가자입니다.
# 기사를 문법 오류, 완전성, 출판 적합성, 과장된 선정성 측면에서 평가하십시오.
# 또한 기사에 사용된 언어가 한국어인지, 단어 수 요건을 충족하는지 확인하십시오.
# 네 가지 이진 점수를 제공하십시오: 
# 기사를 게시할 수 있는지 ('yes' 또는 'no'), 단어 수가 적절한지 ('yes' 또는 'no'), 
# 선정적으로 작성되었는지 ('yes' 또는 'no'), 언어가 한국어인지 ('yes' 또는 'no').
postability_system = """You are a grader assessing whether a news article is ready to be posted, if it meets the minimum word count of 200 words, is written in a sensationalistic style, and if it is in Korean. \n
    Evaluate the article for grammatical errors, completeness, appropriateness for publication, and EXAGERATED sensationalism. \n
    Also, confirm if the language used in the article is Korean and it meets the word count requirement. \n
    Provide four binary scores: one to indicate if the article can be posted ('yes' or 'no'), one for adequate word count ('yes' or 'no'), one for sensationalistic writing ('yes' or 'no'), and another if the language is Korean ('yes' or 'no')."""

# 프롬프트 생성
postability_grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", postability_system),
        ("human", "News Article:\n\n {article}")
    ]
)

# LLM 체인 생성
editor = postability_grade_prompt | structured_llm_postability_grader
# 뉴스 기사 입력
result = editor.invoke(
    {"article": "메시가 레알 마드리드에서 FC 바르셀로나로 이적한다고 보도되었습니다."}
    # {"article": "Es wurde gemeldet, dass Messi von Real Madrid zu FC Barcelona wechselt."}
)
result

### 번역자
- 기사를 한국어로 번역.
- 구조화된 출력 불필요.

LLM 체인 생성
- 외국어를 한국어로 번역

In [ ]:
# 기사 번역 담당 LLM
llm_translation = Get_LLM()

# 시스템 프롬프트:
# 당신은 기사를 한국어로 번역하는 번역가입니다. 원래의 어조와 스타일을 유지하고 존칭을 사용하여 정확하게 번역하십시오.
translation_system = """You are a translator converting articles into Korean. Translate accurately while maintaining the original tone and style, and use honorifics to make it suitable for a news article."""
# translation_system = """You are a translator converting articles into Korean. Translate accurately while maintaining the original tone and style, and use honorifics."""
# translation_system = """You are a translator converting articles into Korean. Translate the text accurately while maintaining the original tone and style."""

# 프롬프트 생성
translation_prompt = ChatPromptTemplate.from_messages(
    [("system", translation_system), ("human", "Article to translate:\n\n {article}")]
)

# LLM 체인 생성
translator = translation_prompt | llm_translation

result = translator.invoke(
    {
        "article": "It has been reported that Messi will transfer from Real Madrid to FC Barcelona."
    }
)
print(result)

### 확장자
- 기사를 200단어 이상으로 확장.
- 짧은 기사를 더 길게 작성

In [ ]:
# 기사 확장 LLM - 200자 이상을 재 작성
llm_expansion = Get_LLM()

# 시스템 프롬프트:
# 당신은 주어진 기사를 최소 200단어로 확장하되, 관련성, 일관성, 원래의 어조를 유지해야 하는 작가입니다.
expansion_system = """You are a writer tasked with expanding the given article to at least 200 words while maintaining relevance, coherence, and the original tone."""

# 프롬프트 생성
expansion_prompt = ChatPromptTemplate.from_messages(
    [("system", expansion_system), ("human", "Original article:\n\n {article}")]
)

# LLM 체인 생성
expander = expansion_prompt | llm_expansion

# 메시가 레알 마드리드에서 FC 바르셀로나로 이적할 것이라는 보도가 있었습니다.
# article_content = "Lionel Messi is reportedly considering a move from Real Madrid to FC Barcelona next season."
article_content = "라이오넬 메시가 레알 마드리드에서 FC 바르셀로나로 이적할 것이라는 보도가 있었습니다."
result = expander.invoke({"article": article_content})
print(result)

## 워크플로우 통합

### 상태 정의
- 기사 평가 상태만을 가진 단순한 타입

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal

# 상태 저장 클래스
class AgentState(TypedDict):
    article_state: str # 기사 심사/평가 상태

### 노드 정의

- 데스크 - 축구 이적 관련 기사 관련성 여부 결과 반환

In [ ]:
# 기사가 축구 이적 관련 기사인지 관련성 평가를 담당하는 node 함수
def get_transfer_news_grade(state: AgentState) -> AgentState:
    print(f"get_transfer_news_grade: Current state: {state}")
    print("Evaluator: Reading article but doing nothing to change it...")
    return state

- 편집자 - 기사 게시 여부 적절성 평가 결과 반환 

In [ ]:
# 뉴스 기사 심사/평가 담당 node 함수
def evaluate_article(state: AgentState) -> AgentState:
    print(f"evaluate_article: Current state: {state}")
    print("News : Reading article but doing nothing to change it...")
    return state

- 번역자 - 기사 내용을 한국어로 번역

In [ ]:
# 뉴스 기사 한국어 번역 node 함수
def translate_article(state: AgentState) -> AgentState:
    print(f"translate_article: Current state: {state}")
    article = state["article_state"]
    result = translator.invoke({"article": article}) # LLM 번역
    state["article_state"] = result.content
    return state

- 확장자 - 기사 내용을 200단어 이상으로 재 작성

In [ ]:
# 뉴스 기사 최소 200자로 작성 node 함수
def expand_article(state: AgentState) -> AgentState:
    print(f"expand_article: Current state: {state}")
    article = state["article_state"]
    result = expander.invoke({"article": article}) # LLM 200자 이상으로 재 작성
    state["article_state"] = result.content
    return state

- 게시자 - 기사 게시 승인

In [ ]:
# 뉴스 기사 게시 승인 node 함수
def publisher(state: AgentState) -> AgentState:
    print(f"publisher: Current state: {state}")
    print("FINAL_STATE in publisher:", state)
    return state

### 조건부 경로 설정

데스크 라우터
- 기사가 관련 있는 경우 편집자 노드로, 그렇지 않으면 종료 노드로 분기

In [ ]:
# 축구 이적관련 뉴스 기사 여부 심사 node 함수
def evaluator_router(state: AgentState) -> Literal["editor", "not_relevant"]:
    article = state["article_state"]
    evaluator = grade_prompt | structured_llm_grader
    result = evaluator.invoke({"article": article}) # 축구 이적 기사 내용 관련성 체크
    print(f"evaluator_router: Current state: {state}")
    print("Evaluator result: ", result)
    if result.binary_score == 'yes': # 축구 이적 기사가 맞음
        return "editor"
    else:
        return "not_relevant" # 축구 이적 기사가 아님

편집자 라우터
- 게시 가능 여부에 따라 번역자, 확장자 또는 게시자로 분기

In [ ]:
# 기사 내용이 게시 기준에 맞는지 여부 평가 node 함수
def editor_router(state: AgentState) -> Literal["translator", "publisher", "expander"]:
    article = state["article_state"]
    result = editor.invoke({"article": article})
    print(f"editor_router: Current state: {state}")
    print("News chef result: ", result)
    if result.can_be_posted == 'yes': # 기사 게시 ok
        return "publisher"
    elif result.is_language_korean == 'yes': # 기사가 한국어로 되어 있음
        if result.meets_word_count == 'no' or result.is_sensationalistic == 'no': # 기사가 200자 이상이 아니거나, 선정적이지 않은 경우
            return "expander"
    return "translator" # 기사가 한국어가 아님

### Node 생성 및 Edge 연결
- 번역자와 확장자는 항상 편집자로 라우팅

Node 생성

In [ ]:
# graph 객체 생성
workflow = StateGraph(AgentState)

# graph node 생성
workflow.add_node("evaluator_node", get_transfer_news_grade) # 축구 이적 기사 관련성 여부 심사 node
workflow.add_node("editor_node", evaluate_article) # 기사 게시 적정성 평가 node
workflow.add_node("translator_node", translate_article) # 기사 내용을 한국어로 번역하는 node
workflow.add_node("expander_node", expand_article) # 기사 내용을 최소 200자로 작성하는 node
workflow.add_node("publisher_node", publisher) # 기사 게시 승인 node

Edge 연결

In [ ]:
# 축구 이적 기사 관련성 여부 체크 및 분기 edge
workflow.add_conditional_edges(
    "evaluator_node",
    evaluator_router, # 기사 관련성 심사 함수
    {
        "editor":"editor_node", # 기사 게시 적정성 평가 node로 분기
        "not_relevant": END # 기사 관련성 없음 - 종료로 분기
    }
)

# 기사 내용 심사/평가 결과 체크 및 분기 edge
workflow.add_conditional_edges(
    "editor_node",
    editor_router, # 기사 내용 심사/평가 함수
    {
        "translator": "translator_node", # 기사 내용 한국어 번역 node로 분기
        "publisher":"publisher_node", # 게시 승인 node로 분기
        "expander": "expander_node" # 기사 내용 최소 200자 작성 node로 분기
    }
)

# graph edge 생성
workflow.add_edge("translator_node","editor_node") # 기사 번역 -> 기사 내용 심사
workflow.add_edge("expander_node","editor_node") # 기사 200자 작성 -> 기사 내용 심사
workflow.add_edge("publisher_node",END) # 기사 게시 승인 -> 종료

컴파일

In [ ]:
workflow.set_entry_point("evaluator_node") # 시작 node - 데스크

# graph 컴파일
app = workflow.compile()

시각화

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph(xray=True).draw_mermaid_png()))
except:
    pass

### 예제 실행

In [ ]:
# 교황이 오늘 스페인을 방문할 것이다. 
# initial_state = {"article_state": "The Pope will visit Spain today"}
initial_state = {"article_state": "아르헨티나 대표팀은 15일 오전 10시(한국시간) 미국 플로리다 주 마이애미의 하드 록 스타디움에서 남미축구연맹(CONMEBOL) 코파 아메리카 2024 결승전에서 콜롬비아 대표팀과 연장전까지 가는 혈투 끝에 1-0으로 승리했다."}
result = app.invoke(initial_state)

print("Final result:", result)

In [ ]:
initial_state = {"article_state": "Messi gonna switch from barca to real madrid"}
# initial_state = {"article_state": "영국 매체 '풋볼 365'는 AI(인공지능)가 예측한 여름 이적시장 결과를 짚으며 손흥민이 스페인 프로축구 프리메라리가 아틀레티코 마드리드로 향할 수 있다는 가능성을 조명했다."}
result = app.invoke(initial_state)

print("Final result:", result)